# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

## ConversationBufferMemory

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [7]:
llm = ChatOpenAI(temperature=0.2, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

# with verbose=True, we can see the conversation with system

In [8]:
conversation.predict(input="Hi, my name is KM")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is KM
AI:

> Finished chain.


"Hello KM, it's nice to meet you. My name is AI. How can I assist you today?"

In [9]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is KM
AI: Hello KM, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'The answer to 1+1 is 2.'

In [12]:
print(memory.buffer)

Human: Hi, my name is KM
AI: Hello KM, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI: Your name is KM, as you mentioned earlier.


In [13]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is KM\nAI: Hello KM, it's nice to meet you. My name is AI. How can I assist you today?\nHuman: What is 1+1?\nAI: The answer to 1+1 is 2.\nHuman: What is my name?\nAI: Your name is KM, as you mentioned earlier."}

In [10]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is KM
AI: Hello KM, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is KM, as you mentioned earlier.'

The way that the langchain stores memory with ConversationBuffereMemory

In [14]:
memory = ConversationBufferMemory()

In [16]:
memory.save_context({"input": "Hi"}, 
                    {"output": "How are you?"})

In [17]:
print(memory.buffer)

Human: Hi
AI: How are you?


In [18]:
memory.load_memory_variables({})

{'history': 'Human: Hi\nAI: How are you?'}

To keep on adding the conversations to memory

In [19]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [20]:
memory.load_memory_variables({})

{'history': 'Human: Hi\nAI: How are you?\nHuman: Not much, just hanging\nAI: Cool'}

Large Language Models are 'Stateless'(LLM models itself does not remember the conversation)
- Each transaction (call to API endpoint) is Independent
- Chatbots appear to have memory by providing the full conversation as context
- The memory storage is used as an input to LLM - So, the input as #of tokens ususally charges
- Langchain provides several kinds of memory to store and accumulate the conversation

## ConversationBufferWindowMemory

Only stores a window of memory 

In [21]:
from langchain.memory import ConversationBufferWindowMemory

In [22]:
memory = ConversationBufferWindowMemory(k=1)               

k = 1 : means we want to remember just one conversational exchange

In [23]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

# only remembers most recent conversation

In [24]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [27]:
llm = ChatOpenAI(temperature=0.5, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [28]:
conversation.predict(input="Hi, my name is KM")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is KM
AI:

> Finished chain.


"Hello KM, it's nice to meet you! My name is OpenAI. How can I assist you today?"

In [29]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is KM
AI: Hello KM, it's nice to meet you! My name is OpenAI. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 is equal to 2.'

In [30]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI:

> Finished chain.


"I'm sorry, I do not have access to your personal information and therefore do not know your name."

As the k was 1, it's not able to recall previous chats

## ConversationTokenBufferMemory

Memory will limit the number of tokens saved, used to save the cost of LLM based on tokens

In [32]:
# !pip install tiktoken

In [33]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [34]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [35]:
memory.load_memory_variables({})

{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

In [51]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=60)
memory.save_context({"input": "How developed Chat GPT?"},
                    {"output": "ChatGPT is an advanced conversational AI developed by OpenAI, designed to engage in natural language conversations with users."})

In [52]:
memory.load_memory_variables({})

{'history': 'Human: How developed Chat GPT?\nAI: ChatGPT is an advanced conversational AI developed by OpenAI, designed to engage in natural language conversations with users.'}

## ConversationSummaryMemory

Memory used to make and store the summary of the conversation made so far with the system

In [55]:
from langchain.memory import ConversationSummaryBufferMemory

In [56]:
# create a long string
paragraph = "ChatGPT is an advanced conversational AI developed by OpenAI, \
        designed to engage in natural language conversations with users. \
        Powered by the GPT architecture, it can understand and \
        generate human-like text across various topics and contexts. \
        ChatGPT learns from vast amounts of text data to continually improve its responses\
        and adapt to diverse conversational styles. With its versatility and ability to understand nuances in language,\
        ChatGPT serves as a helpful tool for communication, customer service, and creative exploration."



In [59]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=70)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is the paragraph above is"}, 
                    {"output": f"{paragraph}"})

In [60]:
memory.load_memory_variables({})

{'history': 'System: The human and AI engage in a casual conversation, with the AI introducing itself as ChatGPT, an advanced conversational AI developed by OpenAI. The AI explains that it is designed to engage in natural language conversations with users, and can understand and generate human-like text across various topics and contexts. ChatGPT learns from vast amounts of text data to continually improve its responses and adapt to diverse conversational styles, making it a helpful tool for communication, customer service, and creative exploration.'}

Create CoversationalChain to make conversation with the LLM

In [61]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [62]:
conversation.predict(input="What would be a good definitation of chat-gpt?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI engage in a casual conversation, with the AI introducing itself as ChatGPT, an advanced conversational AI developed by OpenAI. The AI explains that it is designed to engage in natural language conversations with users, and can understand and generate human-like text across various topics and contexts. ChatGPT learns from vast amounts of text data to continually improve its responses and adapt to diverse conversational styles, making it a helpful tool for communication, customer service, and creative exploration.
Human: What would be a good definitation of chat-gpt?
AI:

> Finished chain.


'ChatGPT is an advanced conversational AI developed by OpenAI that is designed to engage in natural language conversations with users. It can understand and generate human-like text across various topics and contexts, and learns from vast amounts of text data to continually improve its responses and adapt to diverse conversational styles. In short, ChatGPT is a powerful tool for communication, customer service, and creative exploration.'

In [63]:
memory.load_memory_variables({})

{'history': 'System: The human and AI engage in a casual conversation, with the AI introducing itself as ChatGPT, an advanced conversational AI developed by OpenAI. ChatGPT is designed to engage in natural language conversations with users, and can understand and generate human-like text across various topics and contexts. It learns from vast amounts of text data to continually improve its responses and adapt to diverse conversational styles, making it a helpful tool for communication, customer service, and creative exploration. When asked for a definition, the AI reiterates its capabilities and emphasizes that it is a powerful tool for various purposes.'}